In [4]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px

CSV_PATH = Path("Procentuele aandeel BEV_ FCEV en PHEV personenauto's in het wagenpark.csv")

MONTHS_NL = {
    "januari":1, "februari":2, "maart":3, "april":4, "mei":5, "juni":6,
    "juli":7, "augustus":8, "september":9, "oktober":10, "november":11, "december":12
}

def parse_dutch_date(s: str):
    s = str(s).strip().lower()
    m = re.match(r"(\d{1,2})\s+([a-z]+)\s+(\d{4})", s)
    if not m:
        return pd.NaT
    day = int(m.group(1)); month = MONTHS_NL.get(m.group(2)); year = int(m.group(3))
    return pd.Timestamp(year=year, month=month, day=day) if month else pd.NaT

def to_pct_keep(x):
    raw = str(x)
    s = raw.strip().replace(",", ".").replace("%", "")
    try:
        val = float(s)
    except Exception:
        return np.nan
    return val if "%" in raw else (val * 100.0 if val <= 1.0 else val)

def load_last_per_year(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, sep=";", engine="python", encoding="utf-8-sig")

    new_cols = list(df.columns)
    header = df.iloc[0].tolist()
    new_cols[0] = "Datum"
    for i in range(1, len(new_cols)):
        lab = header[i]
        if isinstance(lab, str) and lab.strip():
            new_cols[i] = lab.strip()
    df.columns = new_cols
    df = df.iloc[1:].reset_index(drop=True)


    df["date"] = df["Datum"].apply(parse_dutch_date)
    df["jaar"] = df["date"].dt.year

 
    for col in ["BEV", "FCEV", "PHEV"]:
        df[col] = df[col].apply(to_pct_keep)


    idx_last = df.groupby("jaar")["date"].idxmax()
    last = (df.loc[idx_last, ["jaar", "BEV", "FCEV", "PHEV"]]
              .sort_values("jaar")
              .reset_index(drop=True))
    last["BEV+PHEV+FCEV"] = last[["BEV","PHEV","FCEV"]].sum(axis=1)
    return last[["jaar","BEV+PHEV+FCEV"]]

ts_last = load_last_per_year(CSV_PATH)

ts_last = ts_last[ts_last["jaar"] <= 2023].reset_index(drop=True)


# Plotly figure
fig = px.line(
    ts_last,
    x="jaar",
    y="BEV+PHEV+FCEV",
    markers=True,
    title="Amount(%) of BEV + PHEV + FCEV on the road"
)


fig.update_layout(
    xaxis_title="year",
    yaxis_title="AMount (%)",
    template="plotly_white",
    hovermode="x unified",
)
fig.update_yaxes(range=[0, 20]) 
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Aandeel: %{y:.2f}%<extra></extra>"
)


fig.show()

